In [43]:
#용량이 너무 크니까 5개 씩만 가져와서 따로 저장
IMAGE_PATH = 'Data/IMAGE/'  # 이미지 파일들이 들어있는 경로
JSON_PATH = 'Data/JSON/'  # JSON 파일들이 들어있는 경로
SAVE_PATH = 'Data/metadata/'  # 결과 파일을 저장할 경로

In [44]:
import json
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

chart_data(data) : JSON 파일의 정보를 리스트로 저장

In [45]:
def chart_data(data):
    datatable = []    
    num = len(data)
    for n in range(num):
        title = data[n]['title'] if data[n]['is_title'] else ''
        legend = data[n]['legend'] if data[n]['is_legend'] else ''
        datalabel = data[n]['data_label'] if data[n]['is_datalabel'] else [0]
        unit = data[n]['unit'] if data[n]['is_unit'] else ''
        base = data[n]['base'] if data[n]['is_base'] else ''
        x_axis_title = data[n]['axis_title']['x_axis']
        y_axis_title = data[n]['axis_title']['y_axis']
        x_axis = data[n]['axis_label']['x_axis'] if data[n]['is_axis_label_x_axis'] else [0]
        y_axis = data[n]['axis_label']['y_axis'] if data[n]['is_axis_label_y_axis'] else [0]
        
        if len(legend) > 1:
            datalabel = np.array(datalabel).transpose().tolist()          
                
        datatable.append([title, legend, datalabel, unit, base, x_axis_title, y_axis_title, x_axis, y_axis])
        
    return datatable

datatable(data, chart_type) :    
chart_data()에서 변환된 리스트를 사용할 수 있는 텍스트의 형태로 변환    
(원형 차트만 다룰 것이니 나의 경우 chart_type는 모두 2)

In [46]:
def datatable(data, chart_type):
    data_table = ''
    num = len(data)
    
    #차트가 두 개의 그룹인 경우
    if len(data)  == 2:
        temp = []
        temp.append(f"대상: {data[0][4]}")
        temp.append(f"제목: {data[0][0]}")
        temp.append(f"유형: {' '.join(chart_type[0:2])}")
        temp.append(f"{data[0][5]} | {data[0][1][0]}({data[0][3]}) | {data[1][1][0]}({data[1][3]})")
    
        x_axis = data[0][7]
        for idx, x in enumerate(x_axis):
            temp.append(f"{x} | {data[0][2][0][idx]} | {data[1][2][0][idx]}")
            
        data_table = '\n'.join(temp)

    #차트가 여러 개의 그룹인 경우    
    else:
        for n in range(num):
            temp = []
            
            title, legend, datalabel, unit, base, x_axis_title, y_axis_title, x_axis, y_axis = data[n]
            legend = [element + f"({unit})" for element in legend]
            
            #세로 막대형일 때
            if len(legend) > 1:
                temp.append(f"대상: {base}")
                temp.append(f"제목: {title}")
                temp.append(f"유형: {' '.join(chart_type[0:2])}")
                temp.append(f"{x_axis_title} | {' | '.join(legend)}")
                
                #원형 차트일 때
                if chart_type[2] == "원형":
                    datalabel = sum(datalabel, [])
                    temp.append(f"{' | '.join([str(d) for d in datalabel])}")
                    data_table = '\n'.join(temp)
                
                #가로 막대형일 때
                else:
                    axis = y_axis if chart_type[2] == "가로 막대형" else x_axis
                    for idx, (x, d) in enumerate(zip(axis, datalabel)):
                        temp_d = [str(e) for e in d]
                        temp_d = " | ".join(temp_d)
                        row = f"{x} | {temp_d}"
                        temp.append(row)
                    data_table = '\n'.join(temp)
            
            #나머지 차트 형태일 때
            else:
                temp.append(f"대상: {base}")
                temp.append(f"제목: {title}")
                temp.append(f"유형: {' '.join(chart_type[0:2])}")
                temp.append(f"{x_axis_title} | {unit}")
                
                axis = y_axis if chart_type[2] == "가로 막대형" else x_axis
                datalabel = datalabel[0]
                
                for idx, x in enumerate(axis):
                    row = f"{x} | {str(datalabel[idx])}"
                    temp.append(row)
                data_table = '\n'.join(temp)
    
    return data_table

main(IMAGE_PATH, JSON_PATH, SAVE_PATH): 데이터 파일을 가져오고, 위의 함수들 실행

In [47]:
def main(IMAGE_PATH, JSON_PATH, SAVE_PATH):
    #이미지 파일을 가져옴
    image_file_list = os.listdir(IMAGE_PATH) 
    
    dataset = []
    for image_file in tqdm(image_file_list):
        #이미지 파일에 대응하는 JSON 파일을 가져옴
        data_id = image_file.split(".jpg")[0]
        json_file_name = image_file.replace("Source", "Label").split(".jpg")[0] + ".json"
        
        with open(JSON_PATH + json_file_name, "r") as f:
            json_data = json.load(f)
        
        #차트 유형 정보를 가져옴
        chart_multi = json_data['metadata']['chart_multi']
        chart_main = json_data['metadata']['chart_main']
        chart_sub = json_data['metadata']['chart_sub']
        chart_type = [chart_multi, chart_sub, chart_main]
        
        #라벨링데이터의 설명문과 요약문을 불러옴
        chart_annotations = json_data['annotations']
        description = "설명: " + json_data["description"].strip()
        summary = "요약: " + "\\".join(json_data["summary"])
        texts = description + "\n" + summary
        
        #차트 데이터를 리스트로 불러오기
        try:
            charData = chart_data(chart_annotations)
        except:
            pass
        
        #차트 데이터를 텍스트로 가져옴
        try:
            dataTable = datatable(charData, chart_type)
            dataset.append([image_file, dataTable, texts, data_id])
        except:
            pass
    
    df = pd.DataFrame(dataset, columns=["file_name", "text", "description", "data_id"])
    
    if not os.path.isdir(SAVE_PATH):
        os.mkdir(SAVE_PATH)
    df.to_csv(SAVE_PATH + "metadata.csv", index=False, header=True)
    print("save")

    return df

In [48]:
main(IMAGE_PATH, JSON_PATH, SAVE_PATH)

100%|██████████| 5/5 [00:00<00:00, 5001.55it/s]

save


,file_name,text,description,data_id
0,C_Source_295013_pie_standard.jpg,대상: \n제목: 2016년 표백제에 대한 위험성 인식\n유형: 단일형 원형\n |...,설명: 2016년 표백제에 대한 위험성 인식을 나타낸 원형 그래프이다. 2016년 ...,C_Source_295013_pie_standard
1,C_Source_295014_pie_standard.jpg,대상: \n제목: 2016년 합성세제에 대한 위험성 인식\n유형: 단일형 원형\n ...,설명: 2016년 합성세제에 대한 위험성 인식을 나타낸 원형 그래프이다. 2016년...,C_Source_295014_pie_standard
2,C_Source_295016_pie_standard.jpg,대상: \n제목: SMS등 통한 선거 참여 확대를 위한 선거법 개정의 필요성\n유형...,설명: SNS등 통한 선거 참여 확대를 위한 선거법 개정의 필요성에 대해 나타낸 원...,C_Source_295016_pie_standard
3,C_Source_295018_pie_standard.jpg,대상: \n제목: 위험요소별 경험 유무 : 가정폭력\n유형: 단일형 원형\n | 피...,설명: 경험 유무 중 가정폭력에 대해 나타낸 원형 그래프이다. 경험 유무 중 가정폭...,C_Source_295018_pie_standard
4,C_Source_295885_pie_standard.jpg,대상: \n제목: 스마트폰 사용자의 가정폭력 경험 여부\n유형: 단일형 원형\n |...,설명: 스마트폰 사용자의 가정폭력 경험 여부를 나타낸 원형 그래프이다. 스마트폰 사...,C_Source_295885_pie_standard
